In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad eventos totales por usuario

In [14]:
df = events_df.groupby('person').event.count()

In [15]:
features_df = features_df.join(df, on='person')

In [16]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad eventos de cada uno

In [17]:
df = events_df.groupby('person').event.value_counts()

In [18]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [19]:
df = df.drop(columns=['lead'])

In [20]:
df['ach*bl'] = (df['ad campaign hit'])*(df['brand listing'])
df['ach*gl'] = (df['ad campaign hit'])*(df['generic listing'])
df['ach*seh'] = (df['ad campaign hit'])*(df['search engine hit'])
df['ach*sp'] = (df['ad campaign hit'])*(df['searched products'])
df['ach*vp'] = (df['ad campaign hit'])*(df['viewed product'])
df['ach*vs'] = (df['ad campaign hit'])*(df['visited site'])

In [21]:
features_df = features_df.join(df, on='person')

In [22]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## Quincenas

In [23]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [24]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [25]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [26]:
features_df = features_df.join(df, on='person')

In [101]:
#Tiro las columnas con menos de 0.04 de importance
features_df = features_df.drop(columns=['q3', 'q1', 'q8', 'q7', 'q6', 'q5', 'q4', 'q2', 'q10', 'q9', 'cant_sp', 'cant_cv', 'cant_ach'])

In [8]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [102]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 16)

In [111]:
train_df.fillna(0, inplace=True)

In [103]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 15)

In [110]:
to_predict.fillna(0, inplace=True)

In [104]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [113]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [114]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## SVM

In [7]:
from sklearn.svm import LinearSVR

In [108]:
regr = LinearSVR(random_state=0, tol=1e-5)

In [115]:
regr.fit(X_train, y_train)

/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=0, tol=1e-05, verbose=0)

In [116]:
preds = regr.predict(X_test)

In [117]:
preds

array([ 0.00123695,  0.00624969, -0.00143712, ...,  0.00717431,
        0.10904095,  0.02264238])

In [118]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

0.5360951473367238

In [119]:
from sklearn import linear_model
n_samples, n_features = 10, 5

clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)

/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=1000,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [120]:
preds = clf.predict(X_test)

In [47]:
roc_auc_score(y_test, preds)

0.5134079497975761

In [121]:
roc_auc_score(y_test, preds)

0.4489880425657432

## Random Forest

In [122]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [123]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [124]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [50]:
roc_auc_score(y_test,predictions)

0.790868072775363

In [125]:
roc_auc_score(y_test,predictions)

0.7325265549688665

In [126]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cant_eventos_totales Importance: 0.13
Variable: cant_vp              Importance: 0.11
Variable: cant_ch              Importance: 0.1
Variable: ach*vp               Importance: 0.09
Variable: cant_bl              Importance: 0.07
Variable: ach*vs               Importance: 0.07
Variable: cant_gl              Importance: 0.06
Variable: cant_vs              Importance: 0.06
Variable: ach*bl               Importance: 0.06
Variable: ach*gl               Importance: 0.06
Variable: ach*seh              Importance: 0.06
Variable: cant_seh             Importance: 0.05
Variable: cant_s_p             Importance: 0.05
Variable: ach*sp               Importance: 0.05


In [154]:
rf = RandomForestRegressor()

#Grid search
rf_param_grid = {
    'max_depth':[None],
    'max_features':[1,3,10],
    #'min_sample_split':[2,3],
    'min_samples_leaf':[1,3,10],
    'n_estimators':[50,75,100],
}

rf_gs = GridSearchCV(rf, param_grid=rf_param_grid, scoring='roc_auc', n_jobs=4, refit=True, cv=2, verbose=1, return_train_score=True)

In [155]:
rf_gs.fit(X_train, y_train)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   12.9s
[Parallel(n_jobs=4)]: Done  54 out of  54 | elapsed:   18.7s finished
/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/model_selection/_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'max_depth': [None], 'max_features': [1, 3, 10], 'min_samples_leaf': [1, 3, 10], 'n_estimators': [50, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=1)

In [157]:
rf_gs.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=75, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [158]:
rf_gs.best_score_

0.8076120303608332

In [161]:
preds_rf_gs = rf_gs.predict(to_predict.drop(columns=['person']))

## XGBoost

In [184]:
import xgboost as xgb
from xgboost import XGBRegressor
#Parametros con los que obtuvimos 0.82

xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [186]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [128]:
preds = xg_reg.predict(X_test)

In [58]:
roc_auc_score(y_test,preds)

0.8382091724444185

In [129]:
roc_auc_score(y_test,preds)

0.7963767904547974

In [61]:
from sklearn.model_selection import GridSearchCV

In [73]:
param_grid =  {
        'min_child_weight': [1, 5, 10],
        'gamma': [1, 1.5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3],
        'colsample_bylevel': [0.8, 0.9],
        'max_depth': [9, 11, 13],
        'base_score': [0.1, 0.2],
        'n_estimators': [50, 70, 80, 90]
        }

In [74]:
clf = GridSearchCV(xg_reg, param_grid, scoring='roc_auc', n_jobs=4, refit=True, cv=3, verbose=3, return_train_score=True)
clf.fit(X_train, y_train)

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   10.7s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   45.7s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  7.1min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed:  9.9min
[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed: 13.5min
[Parallel(n_jobs=4)]: Done 2040 tasks      | elapsed: 18.5min
[Parallel(n_jobs=4)]: Done 2584 tasks      | elapsed: 23.0min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 28.8min
[Parallel(n_jobs=4)]: Done 3864 tasks      | elapsed: 36.5min
[Parallel(n_jobs=4)]: Done 3888 out of 3888 | elapsed: 36.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'min_child_weight': [1, 5, 10], 'gamma': [1, 1.5, 10], 'colsample_bytree': [0.1, 0.2, 0.3], 'colsample_bylevel': [0.8, 0.9], 'max_depth': [9, 11, 13], 'base_score': [0.1, 0.2], 'n_estimators': [50, 70, 80, 90]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [76]:
clf.best_estimator_

XGBRegressor(alpha=20, base_score=0.2, booster='dart', colsample_bylevel=0.8,
       colsample_bytree=0.3, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=11,
       min_child_weight=5, missing=None, n_estimators=80, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [77]:
clf.best_score_

0.8442700635802476

In [78]:
preds_gs = clf.predict(X_test)

In [79]:
roc_auc_score(y_test, preds_gs)

0.8357319512108158

In [187]:
preds_xgb = xg_reg.predict(to_predict.drop(columns=['person']))

In [188]:
preds_xgb.shape

(19415,)

In [189]:
preds_xgb

array([0.01059394, 0.0862428 , 0.01435047, ..., 0.10477064, 0.10477064,
       0.10477064], dtype=float32)

In [167]:
preds_rf_gs.shape

(19415,)

In [190]:
combinado_xgb_y_rf = preds_rf_gs*0.2 + preds_xgb*0.8
combinado_xgb_y_rf

array([0.00934372, 0.0930363 , 0.01390681, ..., 0.10277097, 0.10277097,
       0.10277097])

In [170]:
roc_auc_score(y_test, combinado_xgb_y_rf)

ValueError: Found input variables with inconsistent numbers of samples: [3883, 19415]

## Logistic Regression

In [80]:
from sklearn.linear_model import LogisticRegression

In [90]:
params ={
    'C':[0.8, 0.9, 1.0],
    'max_iter':[50, 70, 100],
    'n_jobs':[1, 2],
    'random_state':[1,2,3]
}

In [93]:
l_reg = LogisticRegression(random_state=0, solver='sag')

In [94]:
clf = GridSearchCV(l_reg, params, scoring='roc_auc', n_jobs=4, refit=True, cv=3, verbose=3, return_train_score=True)

In [95]:
clf.fit(X_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   32.6s
[Parallel(n_jobs=4)]: Done 162 out of 162 | elapsed:   44.5s finished
/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='sag',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'C': [0.8, 0.9, 1.0], 'max_iter': [50, 70, 100], 'n_jobs': [1, 2], 'random_state': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [96]:
preds = clf.predict_proba(X_test)[:,1] 

In [97]:
clf.score(X_test, y_test)

0.431318494316248

In [98]:
roc_auc_score(y_test, preds)

0.431318494316248

## CSV a publicar

In [171]:
to_publish = pd.DataFrame()

In [172]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [191]:
to_publish['label'] = combinado_xgb_y_rf
to_publish.shape

(19415, 2)

In [192]:
to_publish.head()

,person,label
0,4886f805,0.009344
1,0297fc1e,0.093036
2,2d681dd8,0.013907
3,cccea85e,0.054946
4,4c8a8b93,0.038813


In [193]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [194]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [195]:
to_publish.to_csv('29_11_2.csv', index=False)